In [ ]:
%%capture
!pip install pytorch_lightning #torch==1.3.1
!pip install transformers
!pip3 -q install emoji


In [ ]:
%matplotlib inline

import json
import logging
from pathlib import Path
import random
import tarfile
import tempfile
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# import pandas_path  # Path style access for pandas
from tqdm import tqdm

import torch                    
import torchvision
# import fasttext
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim

In [ ]:
import ast
import pandas as pd

def load_data_lists(path):
    data_points_lists = []
    with open(path, encoding='utf-8') as f:
        lines = f.readlines()

        for line in lines:
            try:
                data_points_lists.append(ast.literal_eval(line))
            except:
                # Ignore lines with errors
                pass

    print('Found {} lines in "{}".'.format(len(lines), path))
    print('Successfully loaded {} data points from "{}".'.format(len(data_points_lists), path))
    
    return data_points_lists

COLUMN_NAMES = ['ID', 'Text', 'Sarcastic']

def construct_df(data_points_lists, column_names=COLUMN_NAMES):
    df = pd.DataFrame(data_points_lists, columns=column_names)
    df['ID'] = pd.to_numeric(df['ID'])
    df['Sarcastic'] = df['Sarcastic'].astype('bool')
    
    return df

In [ ]:
df_train = construct_df(load_data_lists('train.txt'))

print(df_train.shape)
print(df_train.info())

display(df_train.head())

Found 29040 lines in "train.txt".
Successfully loaded 29040 data points from "train.txt".
(29040, 3)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29040 entries, 0 to 29039
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ID         29040 non-null  int64 
 1   Text       29040 non-null  object
 2   Sarcastic  29040 non-null  bool  
dtypes: bool(1), int64(1), object(1)
memory usage: 482.2+ KB
None


,ID,Text,Sarcastic
0,910308516510011393,most # funny quotes : 21 snarky and # funny qu...,True
1,725333760762363905,spurs # creativethinking ! <url>,True
2,840006160660983809,<user> thanks for showing up for our appointme...,True
3,854334602516733952,only a hardcore fan of sir jonny sins will get...,True
4,908913372199915520,haha . # lol,True


In [ ]:
df_val = construct_df(load_data_lists('valid.txt'), COLUMN_NAMES + ['Sarcastic_human'])
print(df_val.shape)
print(df_val.info())

display(df_val.head())

Found 2410 lines in "valid.txt".
Successfully loaded 2410 data points from "valid.txt".
(2410, 4)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2410 entries, 0 to 2409
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ID               2410 non-null   int64 
 1   Text             2410 non-null   object
 2   Sarcastic        2410 non-null   bool  
 3   Sarcastic_human  2410 non-null   int64 
dtypes: bool(1), int64(2), object(1)
memory usage: 59.0+ KB
None


,ID,Text,Sarcastic,Sarcastic_human
0,915657464401580032,whew ... that extra <num> miles today to the g...,True,1
1,854678856724340736,""" oh , good . now no one will know we 're here...",True,1
2,904892917277274112,how much of it you think is true ? has this be...,True,1
3,855466461296504832,<user> finally found proof that the earth is f...,True,1
4,927373534652805120,many ways to overcome tension & fear but nothi...,True,1


In [ ]:
df_test = construct_df(load_data_lists('test.txt'), COLUMN_NAMES + ['Sarcastic_human'])
print(df_test.shape)
print(df_test.info())

display(df_test.head())

Found 2409 lines in "test.txt".
Successfully loaded 2409 data points from "test.txt".
(2409, 4)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2409 entries, 0 to 2408
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ID               2409 non-null   int64 
 1   Text             2409 non-null   object
 2   Sarcastic        2409 non-null   bool  
 3   Sarcastic_human  2409 non-null   int64 
dtypes: bool(1), int64(2), object(1)
memory usage: 58.9+ KB
None


,ID,Text,Sarcastic,Sarcastic_human
0,862902619928506372,i am guessing # netflix no longer lets you gra...,True,1
1,892551658487631873,it 's the insensitive strikeouts at suntrust p...,True,1
2,853143461360480256,"following the path of the river calder , so .....",True,1
3,918423568823840768,# westernsahara # authority has no lessons 2ge...,True,1
4,731617467718610944,hey <user> great sale !,True,1


In [ ]:
!gdown --id 1dJERrVlp7DlNSXk-uvbbG6Rv7uvqTOKd


Downloading...
From: https://drive.google.com/uc?id=1dJERrVlp7DlNSXk-uvbbG6Rv7uvqTOKd
To: /content/dataset_image.z01
100% 734M/734M [00:04<00:00, 171MB/s]


In [ ]:
!gdown --id 1pODuKC4gP6-QDQonG8XTqI8w8ds68mE3


Downloading...
From: https://drive.google.com/uc?id=1pODuKC4gP6-QDQonG8XTqI8w8ds68mE3
To: /content/dataset_image.z02
100% 734M/734M [00:04<00:00, 172MB/s]


In [ ]:
!gdown --id 1mK0Nf-jv_h2bgHUCRM4_EsdTiiitZ_Uj


Downloading...
From: https://drive.google.com/uc?id=1mK0Nf-jv_h2bgHUCRM4_EsdTiiitZ_Uj
To: /content/dataset_image.z03
100% 734M/734M [00:06<00:00, 115MB/s] 


In [ ]:
!gdown --id 1AOWzlOz5hmdO39dEmzhQ4z_nabgzi7Tu


Downloading...
From: https://drive.google.com/uc?id=1AOWzlOz5hmdO39dEmzhQ4z_nabgzi7Tu
To: /content/dataset_image.zip
100% 460M/460M [00:03<00:00, 142MB/s] 


In [ ]:
%%capture
!zip -FF dataset_image.zip --out dataset_image_full.zip
!unzip -FF dataset_image_full.zip -d /content

In [ ]:
# Create subdirectories
import os
import shutil

train_dir = 'dataset_image/train/'
val_dir = 'dataset_image/val/'
test_dir = 'dataset_image/test/'

os.mkdir(train_dir)
os.mkdir(val_dir)
os.mkdir(test_dir)

In [ ]:
missing_ids = []

for id in df_train['ID']:
    img_path = 'dataset_image/{}.jpg'.format(id)
    if os.path.isfile(img_path):
      shutil.move(img_path, 'dataset_image/train/{}.jpg'.format(id))
    else:
        missing_ids.append(id)

In [ ]:
len(missing_ids)


9224

In [ ]:
df_train = df_train[~df_train['ID'].isin(missing_ids)]
df_train.shape

(19816, 3)

In [ ]:
for id in df_val['ID']:
    img_path = 'dataset_image/{}.jpg'.format(id)
    if os.path.isfile(img_path):
      shutil.move(img_path, 'dataset_image/val/{}.jpg'.format(id))

In [ ]:
for id in df_test['ID']:
    img_path = 'dataset_image/{}.jpg'.format(id)
    if os.path.isfile(img_path):
      shutil.move(img_path, 'dataset_image/test/{}.jpg'.format(id))

In [ ]:
os.mkdir('dataset_image/train/sarc/')
os.mkdir('dataset_image/train/non-sarc/')
os.mkdir('dataset_image/val/sarc/')
os.mkdir('dataset_image/val/non-sarc/')
os.mkdir('dataset_image/test/sarc/')
os.mkdir('dataset_image/test/non-sarc/')

In [ ]:
class_foldername = {
    True: 'sarc',
    False: 'non-sarc'
}

In [ ]:
# Move images to subfolders
for id,label in zip(df_train['ID'], df_train['Sarcastic']):
    shutil.move('dataset_image/train/{}.jpg'.format(id), 'dataset_image/train/{}/{}.jpg'.format(class_foldername[label], id))

In [ ]:
# Move images to subfolders
for id,label in zip(df_val['ID'], df_val['Sarcastic']):
    shutil.move('dataset_image/val/{}.jpg'.format(id), 'dataset_image/val/{}/{}.jpg'.format(class_foldername[label], id))


In [ ]:
# Move images to subfolders
for id,label in zip(df_test['ID'], df_test['Sarcastic']):
    shutil.move('dataset_image/test/{}.jpg'.format(id), 'dataset_image/test/{}/{}.jpg'.format(class_foldername[label], id))

In [ ]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

data_transforms = {
    'train':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize
    ]),
    'validation':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        normalize
    ]),
}

image_datasets = {
    'train': 
    datasets.ImageFolder('dataset_image/train', data_transforms['train']),
    'validation': 
    datasets.ImageFolder('dataset_image/val', data_transforms['validation'])
}

image_dataloaders = {
    'train':
    torch.utils.data.DataLoader(image_datasets['train'],
                                batch_size=64,
                                shuffle=True),
    'validation':
    torch.utils.data.DataLoader(image_datasets['validation'],
                                batch_size=64,
                                shuffle=False)
}

In [ ]:
# dataloader for txt data
from transformers import AutoModel, AutoTokenizer

from sklearn.utils import shuffle
from transformers import (get_linear_schedule_with_warmup,AdamW,AutoModel, AutoTokenizer,
                            AutoModelForSequenceClassification)
from torch.utils.data import (TensorDataset,DataLoader,
                             RandomSampler, SequentialSampler, Dataset)


train_df = shuffle(df_train, random_state=42)
valid_df = shuffle(df_val, random_state=42)
test_df = shuffle(df_test, random_state=42)

In [ ]:
bertweet = AutoModel.from_pretrained("vinai/bertweet-base")
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False)

text_model = AutoModelForSequenceClassification.from_pretrained(
    "vinai/bertweet-base",
        num_labels = 2,  
        output_attentions = False, 
        output_hidden_states = True,
    )

Downloading:   0%|          | 0.00/558 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/517M [00:00<?, ?B/s]

Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/824k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSe

In [ ]:
def bert_encode(df, tokenizer):
    input_ids = []
    attention_masks = []
    # print(df)
    for sent,obj in zip(df[['Text']].values,df["object"].values):
        sent = sent.item()
        #print("sent")
        #print(sent)
        #obj = obj.item()
        #print(obj)
        sent = obj+sent
        # print("combined")
        # print(sent)
        encoded_dict = tokenizer.encode_plus(
                            sent,                      
                            add_special_tokens = True, 
                            max_length = 128,           
                            pad_to_max_length = True,
                            truncation = True,
                            return_attention_mask = True,   
                            return_tensors = 'pt',    
                    )
           
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)

    inputs = {
    'input_word_ids': input_ids,
    'input_mask': attention_masks}

    return inputs

In [ ]:
from torch.utils.data import (TensorDataset,DataLoader,
                             RandomSampler, SequentialSampler, Dataset)

def prepare_dataloaders(train_df,test_df,batch_size=64):
    # Load the AutoTokenizer with a normalization mode if the input Tweet is raw
    
    tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False, normalization=True)
    
    tweet_train = bert_encode(train_df, tokenizer)
    tweet_train_labels = train_df['Sarcastic'].astype(int)
    
    tweet_test = bert_encode(test_df, tokenizer)

    input_ids, attention_masks = tweet_train.values()
    labels = torch.tensor(tweet_train_labels.values)
    train_dataset = TensorDataset(input_ids, attention_masks, labels)

    
    input_ids, attention_masks = tweet_test.values()
    test_dataset = TensorDataset(input_ids, attention_masks)

    
    train_dataloader = DataLoader(
                train_dataset,
                sampler = RandomSampler(train_dataset), 
                batch_size = batch_size 
            )


    test_dataloader = DataLoader(
                test_dataset, 
                sampler = SequentialSampler(test_dataset), 
                batch_size = batch_size
            )
    return train_dataloader, test_dataloader

In [ ]:
#text_train_dataloader,text_test_dataloader = prepare_dataloaders(train_df, test_df)
#_,text_val_dataloader = prepare_dataloaders(train_df, valid_df)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
import pandas as pd

train_objects = pd.read_csv('object_image2.csv', header=None, names=['ID', 'objects'])

train_objects.info()
train_objects.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19816 entries, 0 to 19815
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   ID       19816 non-null  int64 
 1   objects  19816 non-null  object
dtypes: int64(1), object(1)
memory usage: 309.8+ KB


,ID,objects
0,840006160660983809,['tvmonitor']
1,908913372199915520,"['zebra', 'cow']"
2,916496521406726145,['train']
3,916364004129304576,[]
4,853866052589154304,[]


In [ ]:
for _, row in train_objects.iterrows():
    train_df.loc[train_df["ID"] == int(row['ID']),'object'] = row['objects']  

train_df.head()

,ID,Text,Sarcastic,object
2456,818244815624302594,2 win a free pair of tickets to the rockets vs...,False,['bus']
847,709583015987445760,thanks <user> for all the pepper . i really en...,True,['sandwich']
26717,684979497074991104,these things go well together,True,[]
14159,820051629370052608,appreciate this evolution,False,['person']
16161,871151361047445504,this is what lambie is like ? i would never ha...,True,['refrigerator']


In [ ]:
train_df['object'] = train_df['object'].apply(lambda x: x[1:-1])
train_df.head()

,ID,Text,Sarcastic,object
2456,818244815624302594,2 win a free pair of tickets to the rockets vs...,False,'bus'
847,709583015987445760,thanks <user> for all the pepper . i really en...,True,'sandwich'
26717,684979497074991104,these things go well together,True,
14159,820051629370052608,appreciate this evolution,False,'person'
16161,871151361047445504,this is what lambie is like ? i would never ha...,True,'refrigerator'


In [ ]:
val_objects = pd.read_csv('object_image_val.csv', header=None, names=['ID', 'objects'])

val_objects.info()
val_objects.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2410 entries, 0 to 2409
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   ID       2410 non-null   int64 
 1   objects  2410 non-null   object
dtypes: int64(1), object(1)
memory usage: 37.8+ KB


,ID,objects
0,915657464401580032,"['bicycle', 'backpack']"
1,854678856724340736,"['person', 'handbag']"
2,904892917277274112,"['laptop', 'book', 'person']"
3,855466461296504832,['chair']
4,927373534652805120,"['book', 'pottedplant']"


In [ ]:
for _, row in val_objects.iterrows():
    valid_df.loc[valid_df["ID"] == int(row['ID']),'object'] = row['objects']  

valid_df.head()

,ID,Text,Sarcastic,Sarcastic_human,object
2009,819329815048241152,"welcome to the the vitale 's , whose 2 daughte...",False,0,"['person', 'pottedplant', 'bench']"
2082,819692523094470656,im cryif i love you mkre than anything in the ...,False,0,['person']
1684,820054764733931520,the beauty of nature,False,0,[]
2301,820052946788515845,i just choked,False,0,['book']
792,794312091402436608,yup . nearly an hour later . moved about <num>...,True,1,"['laptop', 'person']"


In [ ]:
valid_df['object'] = valid_df['object'].apply(lambda x: x[1:-1])
valid_df.head()

,ID,Text,Sarcastic,Sarcastic_human,object
2009,819329815048241152,"welcome to the the vitale 's , whose 2 daughte...",False,0,"'person', 'pottedplant', 'bench'"
2082,819692523094470656,im cryif i love you mkre than anything in the ...,False,0,'person'
1684,820054764733931520,the beauty of nature,False,0,
2301,820052946788515845,i just choked,False,0,'book'
792,794312091402436608,yup . nearly an hour later . moved about <num>...,True,1,"'laptop', 'person'"


In [ ]:
test_objects = pd.read_csv('object_image_test.csv', header=None, names=['ID', 'objects'])

test_objects.info()
test_objects.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2409 entries, 0 to 2408
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   ID       2409 non-null   int64 
 1   objects  2409 non-null   object
dtypes: int64(1), object(1)
memory usage: 37.8+ KB


,ID,objects
0,862902619928506372,[]
1,892551658487631873,['person']
2,853143461360480256,['cat']
3,918423568823840768,"['traffic light', 'book']"
4,731617467718610944,"['laptop', 'person', 'bottle']"


In [ ]:
for _, row in test_objects.iterrows():
    test_df.loc[test_df["ID"] == int(row['ID']),'object'] = row['objects']  

test_df.head()

,ID,Text,Sarcastic,Sarcastic_human,object
2009,822947223528869888,our signs in action at # womensmarchsanjose .,False,0,"['person', 'backpack']"
2082,822223962511843328,"makeup i did on snap today , details will be l...",False,0,['person']
1684,822589093301391363,while in the workshop cleaning found the oppos...,False,0,['knife']
2301,819323165662965762,"best 10 minutes of my day , god is good",False,0,[]
792,875868378698797056,it was nice enjoying this cooler weather for a...,True,1,[]


In [ ]:
test_df['object'] = test_df['object'].apply(lambda x: x[1:-1])
test_df.head()

,ID,Text,Sarcastic,Sarcastic_human,object
2009,822947223528869888,our signs in action at # womensmarchsanjose .,False,0,"'person', 'backpack'"
2082,822223962511843328,"makeup i did on snap today , details will be l...",False,0,'person'
1684,822589093301391363,while in the workshop cleaning found the oppos...,False,0,'knife'
2301,819323165662965762,"best 10 minutes of my day , god is good",False,0,
792,875868378698797056,it was nice enjoying this cooler weather for a...,True,1,


In [ ]:
# import csv

# with open('object_image2.csv') as csv_file:
#     csv_reader = csv.reader(csv_file, delimiter=',')
#     for row in csv_reader:
#         ID = row[0]
#         obj = row[1]
        
#         str_obj = obj.replace("[","")
#         str_obj = str_obj.replace("]","")
#         train_df.loc[train_df["ID"] == int(row[0]),"object"] = str_obj        

In [ ]:

#print(df_train["object"])
print(train_df.isna().sum())
print(valid_df.isna().sum())
print(test_df.isna().sum())
# train_df = train_df.fillna(' ')
# print(train_df["object"])

ID           0
Text         0
Sarcastic    0
object       0
dtype: int64
ID                 0
Text               0
Sarcastic          0
Sarcastic_human    0
object             0
dtype: int64
ID                 0
Text               0
Sarcastic          0
Sarcastic_human    0
object             0
dtype: int64


In [ ]:
text_train_dataloader,text_test_dataloader = prepare_dataloaders(train_df, test_df)
_,text_val_dataloader = prepare_dataloaders(train_df, valid_df)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2218: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
def prepare_model(model_class="vinai/bertweet-base",num_classes=2,model_to_load=None,total_steps=-1):
    
    #model = AutoModel.from_pretrained(model_class)

    model = AutoModelForSequenceClassification.from_pretrained(
        model_class,
        num_labels = num_classes,  
        output_attentions = False, 
        output_hidden_states = True,
    )

    optimizer = AdamW(model.parameters(),
                    lr = 5e-5,
                    eps = 1e-8
                    )
    scheduler = get_linear_schedule_with_warmup(optimizer, 
                                                num_warmup_steps = 0, 
                                                num_training_steps = total_steps)

    if model_to_load is not None:
        try:
            model.roberta.load_state_dict(torch.load(model_to_load))
            print("LOADED MODEL")
        except:
            pass
    return model, optimizer, scheduler

In [ ]:
# count = 0
# for batch in text_train_dataloader:
#     count+=1
#     print(batch[0])
#     if count==5:
#         break

In [ ]:
class MultiModel(nn.Module):
    def __init__(self):
        super(MultiModel, self).__init__()
        #Resnet50 for image
        self.text_model, _, _ = prepare_model("vinai/bertweet-base" ,num_classes=2, model_to_load=None, total_steps = total_steps)
        self.linear_1 = nn.Linear(768,300)
        #self.text_model = AutoModelForSequenceClassification.from_pretrained(
        #model_class = "vinai/bertweet-base",
        #num_labels = 2,  
        #output_attentions = False, 
        #output_hidden_states = False,)
        
        self.image_model = torchvision.models.resnet50(
            pretrained=True
        )
        #resnet_output_feature_dim = 300
        #bert_output_feature_dim = 300
        #fusion_input_feature = 300+300 = 600
        self.image_model.fc = nn.Sequential(
               nn.Linear(2048, 128),
               nn.ReLU(inplace=True),
               nn.Linear(128, 300))
        
        
        # Combined
        
        #fusion_input_feature = 300+300 = 600
        #fusion_output_size = 128
        self.combined_fc1 = nn.Linear(600, 256)
        self.combined_fc2 = nn.Linear(256, 128)
        
        self.output_fc = nn.Linear(128, 2)
       
        #self.text_train_dataset = image_dataloaders["train"]
        #self.text_dev_dataset = image_dataloaders["validation"]
        #self.image_train_dataset = text_train_dataloader
        #self.image_dev_dataset = text_val_dataloader

    
    def forward(self, text,text_mask,image):
        text = text
        bert_out = self.text_model(text)
        #last_layer = bert_out.last_hidden_state[:,0,:]
        #last_layer = self.linear_1(last_layer)
        #print(len(bert_out))
        
        last_layer = bert_out.hidden_states[-1][:, 0, :]
        #print(last_layer.shape)
        image = image
        resnet_out = self.image_model(image)
        # print("resnet output")
        # print(resnet_out.shape)
        last_layer= self.linear_1(last_layer)
        combined_emb = torch.cat((resnet_out, last_layer), 1)
        # print("combined_emb")
        # print(combined_emb.shape)
        x_comb = F.relu(self.combined_fc1(combined_emb))
        x_comb = F.relu(self.combined_fc2(x_comb))
        out = torch.sigmoid(self.output_fc(x_comb))
        #logits = self.output_fc(x_comb)
        #pred = torch.nn.functional.softmax(logits)
        return out


In [ ]:
#count = 0
#for batch, [image,label] in zip(text_train_dataloader,image_dataloaders["train"]):
#    count+=1
#    print("text input shape")
#    print(batch[0].shape)
#    print(batch[1].shape)
#    print(batch[2].shape)
#    print("image input shape")
#    print(image.shape)
#    print(label.shape)
#    if count==5:
#        break

In [ ]:
import gc

def train_model(model, dataloaders, criterion, optimizer, num_epochs):
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        for phase in ['train', 'validation']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for image_inputs,text_inputs,text_masks,labels in dataloaders[phase]:
                if phase != 'train':
                    with torch.no_grad():
                        text_inputs = text_inputs.to(device)
                        image_inputs = image_inputs.to(device)
                        labels = labels.to(device)

                        outputs = model(text_inputs,text_masks,image_inputs)
                        
                        loss = criterion(outputs, labels)
                        if phase == 'train':
                            optimizer.zero_grad()
                            loss.backward()
                            optimizer.step()

                        _, preds = torch.max(outputs.detach(), 1)
                        running_loss += loss.detach().item() * image_inputs.size(0)
                        running_corrects += torch.sum(preds == labels.data)
                else:
                    text_inputs = text_inputs.to(device)
                    image_inputs = image_inputs.to(device)
                    labels = labels.to(device)

                    outputs = model(text_inputs,text_masks,image_inputs)
                    
                    loss = criterion(outputs, labels)
                    if phase == 'train':
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()

                    _, preds = torch.max(outputs.detach(), 1)
                    running_loss += loss.detach().item() * image_inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(image_datasets[phase])
            epoch_acc = running_corrects.double() / len(image_datasets[phase])

            print('{} loss: {:.4f}, acc: {:.4f}'.format(phase,
                                                        epoch_loss,
                                                        epoch_acc))
            # print('Memory cached: {}'.format(torch.cuda.memory_cached()))
            # print('-' * 10)
            torch.cuda.empty_cache()
            gc.collect()

    return model

In [ ]:
torch.cuda.empty_cache()
gc.collect()

100

In [ ]:
from transformers import AutoModel, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False, normalization=True)

tweet_train = bert_encode(train_df, tokenizer)

input_ids, attention_masks = tweet_train.values()
input_ids = list(input_ids)
attention_masks = list(attention_masks)

tweet_train_dict = {
    id: (ids, masks) for id, ids, masks in zip(train_df['ID'], input_ids, attention_masks)
}

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2218: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
tweet_valid = bert_encode(valid_df, tokenizer)

input_ids, attention_masks = tweet_valid.values()

input_ids = list(input_ids)
attention_masks = list(attention_masks)

tweet_valid_dict = {
    id: (ids, masks) for id, ids, masks in zip(valid_df['ID'], input_ids, attention_masks)
}

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2218: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
class ImageTextDataset(datasets.ImageFolder):
    def __init__(self, img_path, data_transforms, text_data):
        super().__init__(img_path, data_transforms)
        self.text_data = text_data

    def __getitem__(self, index):
        # Image data
        img, label = super(ImageTextDataset, self).__getitem__(index)
        
        # Image ID
        id = int(os.path.basename(self.imgs[index][0]).split('.')[0])

        # Text data
        text_data_tuple = self.text_data[id]
        
        return ((img,)  + text_data_tuple + (label,))

In [ ]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

data_transforms = {
    'train':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize
    ]),
    'validation':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        normalize
    ]),
}

image_datasets = {
    'train': 
    ImageTextDataset('dataset_image/train', data_transforms['train'], tweet_train_dict),
    'validation': 
    ImageTextDataset('dataset_image/val', data_transforms['validation'], tweet_valid_dict)
}


BATCH_SIZE = 64
dataloaders = {
    'train':
    torch.utils.data.DataLoader(image_datasets['train'],
                                batch_size=BATCH_SIZE,
                                shuffle=True),
    'validation':
    torch.utils.data.DataLoader(image_datasets['validation'],
                                batch_size=BATCH_SIZE,
                                shuffle=False)
}

In [ ]:
tweet_test = bert_encode(test_df, tokenizer)

input_ids, attention_masks = tweet_test.values()

input_ids = list(input_ids)
attention_masks = list(attention_masks)

tweet_test_dict = {
    id: (ids, masks) for id, ids, masks in zip(test_df['ID'], input_ids, attention_masks)
}

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2218: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
image_datasets['test'] = ImageTextDataset('dataset_image/test', data_transforms['validation'], tweet_test_dict)

In [ ]:
dataloaders['test'] = torch.utils.data.DataLoader(image_datasets['test'],
                                batch_size=BATCH_SIZE,
                                shuffle=False)

In [ ]:
epochs = 10
total_steps = len(train_df) * epochs
model = MultiModel().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(),
                    lr = 5e-5,
                    eps = 1e-8
                    )

#loss_func = nn.MSELoss()
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.decoder.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-base and are newly initialized: 

  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [ ]:
model_trained = train_model(model,dataloaders, criterion, optimizer, num_epochs=10)

Epoch 1/10
----------
train loss: 0.5868, acc: 0.7104
validation loss: 0.5766, acc: 0.7224
Epoch 2/10
----------
train loss: 0.5473, acc: 0.7537
validation loss: 0.5663, acc: 0.7307
Epoch 3/10
----------
train loss: 0.5252, acc: 0.7771
validation loss: 0.5712, acc: 0.7286
Epoch 4/10
----------
train loss: 0.5125, acc: 0.7930
validation loss: 0.5720, acc: 0.7286
Epoch 5/10
----------
train loss: 0.4997, acc: 0.8064
validation loss: 0.5672, acc: 0.7349
Epoch 6/10
----------
train loss: 0.4844, acc: 0.8206
validation loss: 0.5780, acc: 0.7212
Epoch 7/10
----------
train loss: 0.4702, acc: 0.8366
validation loss: 0.5759, acc: 0.7266
Epoch 8/10
----------
train loss: 0.4648, acc: 0.8430
validation loss: 0.5772, acc: 0.7270
Epoch 9/10
----------
train loss: 0.4520, acc: 0.8555
validation loss: 0.5757, acc: 0.7270
Epoch 10/10
----------
train loss: 0.4503, acc: 0.8583
validation loss: 0.5899, acc: 0.7120


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
torch.save(model, '/content/drive/MyDrive/CSE8803_DLT/yolo_res_bert_multimodal_10epoch.pt')

In [ ]:
%ls /content/drive/MyDrive/CSE8803_DLT/

berttweet_2epoch.pt    Resnet50_10epoch.pt
multimodal_epoch10.pt  yolo_res_bert_multimodal_10epoch.pt


In [ ]:
%ls -la /content/drive/MyDrive/CSE8803_DLT/yolo_res_bert_multimodal_10epoch.pt

-rw------- 1 root root 636958487 Dec  6 07:17 /content/drive/MyDrive/CSE8803_DLT/yolo_res_bert_multimodal_10epoch.pt


## Test

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
m = torch.load('/content/drive/MyDrive/CSE8803_DLT/yolo_res_bert_multimodal_10epoch.pt')
m

MultiModel(
  (text_model): RobertaForSequenceClassification(
    (roberta): RobertaModel(
      (embeddings): RobertaEmbeddings(
        (word_embeddings): Embedding(64001, 768, padding_idx=1)
        (position_embeddings): Embedding(130, 768, padding_idx=1)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): RobertaEncoder(
        (layer): ModuleList(
          (0): RobertaLayer(
            (attention): RobertaAttention(
              (self): RobertaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): RobertaSelfOutput(
                (dense): Linear

In [ ]:
def predict(model,test_dataloader):
    model.eval()
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.to(device)
    preds = []
    true_labels = []

    for image_inputs,text_inputs,text_masks,labels in test_dataloader:
        text_inputs = text_inputs.to(device)
        image_inputs = image_inputs.to(device)
        labels = labels.to(device)
  
        with torch.no_grad():        
            outputs = model(text_inputs,text_masks,image_inputs)

        logits = outputs.detach().cpu().numpy()
        for logit,l in zip(logits, labels):
            preds.append(logit)
            true_labels.append(l)

    return np.array(preds), true_labels

In [ ]:
test_preds, test_labels = predict(m, dataloaders['test'])
test_preds.shape

(2409, 2)

In [ ]:
pd.Series([l.detach().item() for l in test_labels]).value_counts()

0    1404
1    1005
dtype: int64

In [ ]:
test_labels_det = [l.detach().item() for l in test_labels]
len(test_labels_det)

2409

In [ ]:
test_preds[0].argmax()

0

In [ ]:
test_preds_l = np.argmax(test_preds, axis=1)
test_preds_l

array([0, 0, 0, ..., 1, 1, 1])

In [ ]:
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, precision_score, recall_score
        
print('Test data - Accuracy: {:.3f}, Balanced accuracy: {:.3f}, F1: {:.3f}, Precision: {:.3f}, Recall: {:.3f}'
              .format(accuracy_score(test_labels_det, test_preds_l),
                      balanced_accuracy_score(test_labels_det, test_preds_l),
                      f1_score(test_labels_det, test_preds_l),
                      precision_score(test_labels_det, test_preds_l),
                      recall_score(test_labels_det, test_preds_l)))

Test data - Accuracy: 0.743, Balanced accuracy: 0.743, F1: 0.707, Precision: 0.675, Recall: 0.742
